# 🚀 Freqtrade策略测试器

## 📋 教程概述

这是一个通用的Freqtrade策略测试工具，可以帮助你：
- ✅ 验证策略的基本功能
- ✅ 测试参数优化配置
- ✅ 检查数据加载状态
- ✅ 分析策略表现
- ✅ 生成可视化报告

**适用场景：**
- 策略开发完成后验证
- 参数优化前的基础检查
- 快速测试多个策略
- 策略性能对比分析

---

## 🎯 使用前准备

### 1. 环境要求
- ✅ Freqtrade已正确安装
- ✅ 策略文件已创建
- ✅ 配置文件已准备

### 2. 文件结构
```
user_data/
├── strategies/
│   ├── YourStrategy.py
│   └── YourStrategy.json (可选)
├── hyperopts/
│   └── YourHyperOptLoss.py (可选)
└── config.json
```

---

## 🔧 配置参数

请根据你的实际情况修改以下参数：

In [ ]:
# ==============================
# 🔧 用户配置参数
# ==============================

# 策略名称（不含.py后缀）
STRATEGY_NAME = "DoubleMAStrategy"  # 修改为你的策略名称

# 配置文件路径
CONFIG_FILE = "user_data/config_double_ma.json"  # 修改为你的配置文件

# Hyperopt Loss类名称（可选）
HYPEROPT_LOSS_NAME = "DoubleMAHyperOptLoss"  # 修改为你的Loss类名

# 测试模式
TEST_MODE = "full"  # 可选: "basic", "loss", "data", "full"

# ==============================
# 🔧 系统配置（通常无需修改）
# ==============================

import sys
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# 添加项目路径
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# 导入必要的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# 设置matplotlib样式
plt.style.use('default')
sns.set_palette("husl")

print("✅ 环境配置完成")
print(f"📁 当前工作目录: {project_root}")
print(f"🎯 测试策略: {STRATEGY_NAME}")
print(f"⚙️ 配置文件: {CONFIG_FILE}")

## 🔍 策略基础功能测试

这一步将验证策略是否能正确加载和运行。

In [ ]:
# ==============================
# 🔍 策略基础功能测试
# ==============================

def test_strategy_basic_functionality():
    """
    测试策略的基本功能
    """
    print("🔍 测试策略基本功能...")
    
    try:
        from freqtrade.strategy import IStrategy
        from freqtrade.resolvers import StrategyResolver
        from freqtrade.configuration import Configuration
        
        # 加载策略
        config = Configuration.from_files([CONFIG_FILE])
        strategy = StrategyResolver.load_strategy(config)
        
        print("✅ 策略加载成功!"        print(f"   📄 策略名称: {strategy.__class__.__name__}")
        print(f"   ⏰ 时间周期: {strategy.timeframe}")
        print(f"   📈 允许做空: {strategy.can_short}")
        print(f"   🛡️ 止损设置: {strategy.stoploss}")
        print(f"   🎯 最小收益目标: {strategy.minimal_roi}")
        
        # 获取可优化参数
        print("\n📊 可优化参数:")
        
        # 扫描所有Parameter属性
        param_count = 0
        for attr_name in dir(strategy):
            attr = getattr(strategy, attr_name)
            if hasattr(attr, 'value') and hasattr(attr, 'low'):
                param_count += 1
                print(f"   🔧 {attr_name}: {attr.value} (范围: {attr.low}-{attr.high})")
        
        if param_count == 0:
            print("   ⚠️  未找到可优化参数")
        else:
            print(f"\n✅ 找到 {param_count} 个可优化参数")
        
        return True, strategy
        
    except Exception as e:
        print(f"❌ 策略测试失败: {e}")
        print("\n🔧 故障排除建议:")
        print("   1. 检查策略文件是否存在")
        print("   2. 检查策略类名是否正确")
        print("   3. 检查配置文件路径是否正确")
        print("   4. 确认Freqtrade环境已正确安装")
        return False, None

# 执行测试
if TEST_MODE in ["basic", "full"]:
    success, strategy_instance = test_strategy_basic_functionality()
    if not success:
        print("\n🛑 请修复上述问题后再继续")
else:
    print("⏭️ 跳过基础功能测试")
    strategy_instance = None

## 📊 Hyperopt Loss函数测试

测试自定义的优化损失函数是否工作正常。

In [ ]:
# ==============================
# 📊 Hyperopt Loss函数测试
# ==============================

def test_hyperopt_loss():
    """
    测试Hyperopt Loss函数
    """
    print("📊 测试Hyperopt Loss函数...")
    
    try:
        # 动态导入Loss类
        module_path = f"user_data.hyperopts.{HYPEROPT_LOSS_NAME}"
        module = __import__(module_path, fromlist=[HYPEROPT_LOSS_NAME])
        loss_class = getattr(module, HYPEROPT_LOSS_NAME)
        
        print(f"✅ Loss类加载成功: {loss_class.__name__}")
        
        # 创建模拟的交易结果数据
        mock_results = pd.DataFrame({
            'profit_ratio': [0.02, -0.01, 0.03, 0.01, -0.005, 0.025, 0.015, -0.02],
            'trade_duration': [120, 180, 90, 240, 60, 150, 200, 80],
            'max_drawdown': [0.05, 0.08, 0.03, 0.06, 0.04, 0.07, 0.09, 0.02]
        })
        
        print(f"📈 模拟交易数据: {len(mock_results)} 笔交易")
        print(f"   💰 总利润: {mock_results['profit_ratio'].sum():.4f}")
        print(f"   📊 胜率: {(mock_results['profit_ratio'] > 0).mean():.2%}")
        
        # 测试Loss函数
        loss_value = loss_class.hyperopt_loss_function(
            results=mock_results,
            trade_count=len(mock_results),
            min_date=pd.Timestamp('2024-01-01'),
            max_date=pd.Timestamp('2024-01-10'),
            config={},
            processed={}
        )
        
        print("✅ Loss函数计算成功!"        print(f"   🎯 Loss值: {loss_value:.6f}")
        print(f"   📝 Loss值越小越好 (当前: {'优秀' if loss_value < 0.5 else '一般' if loss_value < 1.0 else '需要改进'})")
        
        return True
        
    except ImportError:
        print(f"⚠️  未找到Loss类: {HYPEROPT_LOSS_NAME}")
        print("   💡 如果你没有自定义Loss函数，这是正常的")
        return False
    except Exception as e:
        print(f"❌ Loss函数测试失败: {e}")
        return False

# 执行测试
if TEST_MODE in ["loss", "full"]:
    loss_success = test_hyperopt_loss()
else:
    print("⏭️ 跳过Loss函数测试")
    loss_success = None

## 📈 数据加载状态检查

检查是否有足够的历史数据用于测试。

In [ ]:
# ==============================
# 📈 数据加载状态检查
# ==============================

def test_data_loading():
    """
    测试数据加载功能
    """
    print("📈 检查数据加载状态...")
    
    try:
        # 检查数据目录
        data_dir = project_root / "user_data" / "data"
        if not data_dir.exists():
            print("❌ 未找到数据目录")
            print("   💡 请先运行数据下载命令")
            return False, 0
        
        # 查找可用的数据文件
        data_files = list(data_dir.glob("**/*.feather"))
        if not data_files:
            print("❌ 未找到数据文件")
            print("   💡 请先运行数据下载命令")
            return False, 0
        
        print(f"✅ 找到 {len(data_files)} 个数据文件")
        
        # 分析数据文件
        total_records = 0
        pair_info = []
        
        for file_path in data_files:
            try:
                # 读取数据文件信息
                df = pd.read_feather(file_path)
                record_count = len(df)
                total_records += record_count
                
                # 提取交易对信息
                pair_name = file_path.stem.split('_')[0]  # 从文件名提取交易对
                
                pair_info.append({
                    'pair': pair_name,
                    'records': record_count,
                    'file': file_path.name
                })
                
            except Exception as e:
                print(f"⚠️  无法读取文件 {file_path.name}: {e}")
        
        # 显示数据统计
        print("\n📊 数据统计:"        print(f"   📁 总数据文件: {len(data_files)} 个")
        print(f"   📈 总数据条数: {total_records:,} 条")
        print(f"   📅 平均每文件: {total_records/len(data_files):.0f} 条")
        
        # 显示前5个交易对的信息
        print("\n📋 交易对详情 (前5个):"        for i, info in enumerate(sorted(pair_info, key=lambda x: x['records'], reverse=True)[:5]):
            print(f"   {i+1}. {info['pair']}: {info['records']:,} 条记录")
        
        # 数据充足性检查
        min_records = 1000  # 最少需要的记录数
        sufficient_pairs = sum(1 for info in pair_info if info['records'] >= min_records)
        
        if sufficient_pairs >= 3:
            print(f"\n✅ 数据充足: {sufficient_pairs} 个交易对有足够数据")
        else:
            print(f"\n⚠️  数据可能不足: 只有 {sufficient_pairs} 个交易对有足够数据")
            print(f"   💡 建议至少准备3个交易对的数据")
        
        return True, total_records
        
    except Exception as e:
        print(f"❌ 数据检查失败: {e}")
        return False, 0

# 执行测试
if TEST_MODE in ["data", "full"]:
    data_success, total_records = test_data_loading()
else:
    print("⏭️ 跳过数据检查")
    data_success = None
    total_records = 0

## 📊 测试结果汇总

查看所有测试的综合结果。

In [ ]:
# ==============================
# 📊 测试结果汇总
# ==============================

def generate_test_report():
    """
    生成测试报告
    """
    print("📊 生成测试报告...")
    print("=" * 50)
    
    # 测试结果统计
    tests_run = []
    tests_passed = 0
    
    # 基础功能测试结果
    if TEST_MODE in ["basic", "full"]:
        tests_run.append("基础功能测试")
        if 'success' in locals() and success:
            tests_passed += 1
            print("✅ 基础功能测试: 通过")
        else:
            print("❌ 基础功能测试: 失败")
    
    # Loss函数测试结果
    if TEST_MODE in ["loss", "full"]:
        tests_run.append("Loss函数测试")
        if 'loss_success' in locals() and loss_success:
            tests_passed += 1
            print("✅ Loss函数测试: 通过")
        else:
            print("⚠️  Loss函数测试: 未运行或失败")
    
    # 数据检查结果
    if TEST_MODE in ["data", "full"]:
        tests_run.append("数据检查")
        if 'data_success' in locals() and data_success:
            tests_passed += 1
            print("✅ 数据检查: 通过")
            print(f"   📊 数据总量: {total_records:,} 条记录")
        else:
            print("❌ 数据检查: 失败")
    
    print("=" * 50)
    
    # 总体评估
    total_tests = len(tests_run)
    if total_tests > 0:
        success_rate = tests_passed / total_tests
        print(f"🎯 测试完成率: {tests_passed}/{total_tests} ({success_rate:.1%})")
        
        if success_rate >= 0.8:
            print("🎉 恭喜！策略测试基本通过")
            print("   🚀 可以进行回测或参数优化了")
        elif success_rate >= 0.5:
            print("⚠️  策略测试部分通过")
            print("   🔧 请检查失败的项目并修复")
        else:
            print("❌ 策略测试失败较多")
            print("   🛠️  请仔细检查配置和代码")
    else:
        print("⚠️  未运行任何测试")
    
    # 下一步建议
    print("\n💡 下一步建议:")
    if 'success' in locals() and success:
        print("   1. 📈 运行完整回测")
        print("   2. 🎯 执行参数优化")
        print("   3. 📊 分析策略表现")
    else:
        print("   1. 🔧 修复测试失败的项目")
        print("   2. 📖 检查文档和示例代码")
        print("   3. 💬 在社区寻求帮助")

# 生成报告
generate_test_report()

## 🎯 快速开始命令

基于测试结果，这里是下一步的操作命令：

In [ ]:
# ==============================
# 🎯 快速开始命令
# ==============================

# 1. 基础回测
print("📈 基础回测命令:")
print(f"freqtrade backtesting --config {CONFIG_FILE} --strategy {STRATEGY_NAME} --timerange 20230601-20231231 --dry-run-wallet 10000")

# 2. 参数优化
print("\n🎯 参数优化命令:")
print(f"freqtrade hyperopt --config {CONFIG_FILE} --hyperopt-loss {HYPEROPT_LOSS_NAME} --strategy {STRATEGY_NAME} --epochs 50 --timerange 20230601-20231231 --spaces buy")

# 3. 查看优化结果
print("\n📊 查看优化结果:")
print(f"freqtrade hyperopt-show --hyperopt-loss {HYPEROPT_LOSS_NAME} --strategy {STRATEGY_NAME} --best")

# 4. 使用优化参数进行回测
print("\n🚀 使用优化参数回测:")
print(f"freqtrade backtesting --config {CONFIG_FILE} --strategy {STRATEGY_NAME} --timerange 20230601-20231231 --dry-run-wallet 10000")

print("\n💡 提示: 复制上面的命令在终端中运行")

## 📚 使用教程

### 🔧 如何使用这个Notebook

1. **修改配置参数**: 在第二步中修改策略名称和配置文件路径
2. **选择测试模式**: 设置 `TEST_MODE` 为 "basic", "loss", "data", 或 "full"
3. **运行测试**: 从上到下依次运行所有代码单元
4. **查看结果**: 根据测试结果进行下一步操作

### 🎯 测试模式说明

- **"basic"**: 只测试策略基础功能
- **"loss"**: 只测试Hyperopt Loss函数
- **"data"**: 只检查数据加载状态
- **"full"**: 运行所有测试（推荐）

### 🚨 常见问题解决

#### 问题1: 策略加载失败
**原因**: 策略文件不存在或类名错误
**解决**: 
- 检查 `user_data/strategies/` 目录
- 确认策略类继承自 `IStrategy`

#### 问题2: Loss函数未找到
**原因**: 未创建自定义Loss函数
**解决**: 使用默认Loss函数或创建自定义Loss

#### 问题3: 数据文件缺失
**原因**: 未下载历史数据
**解决**: 运行数据下载命令

### 🎉 成功标志

当你看到以下输出时，说明测试成功：
- ✅ 策略加载成功
- ✅ Loss函数测试成功
- ✅ 数据检查通过
- 🎉 恭喜！策略测试基本通过

---

**🎊 祝你量化交易顺利！**